# DSSA 
## Robert Tarasevič TSF–17

## 2 lab. darbas | Logistinės regresijos modelio parametrų vertinimas
Duomenų faile ksimpah.csv yra realaus medicininio tyrimo duomenys apie paauglių
susirgimą pirminę arterinę hipertenziją. Kategoriniai dvireikšmiai kintamieji:

- Hipertenzija -- ar pacientas serga pirmine arterine hipertenzija;
- Lytis -- paciento lytis;
- Tabakas -- nurodo, ar pacientas rūko;
- Alkoholis -- nurodo, ar pacientas vartoja alkoholį;
- Antsvoris -- nurodo, ar pacientas turi antsvorį.

Taip pat yra keletas kiekybinių kintamųjų:

- Ūgis,
- Svoris,
- KMI -- kūno masės indeksas.


Reikia pasirinkti dvireikšmį kintamąjį, kuris bus atsako kintamasis Y ir jam
parinkti regresorių. Tyrime buvo tiriama hipertenzijos priklausomybė nuo kitų
faktorių, pavyzdžiui, antsvorio, lyties ir pan., todėl Y gali būti Hipertenzija,
o X gali būti Antsvoris. Galimi ir kiti kintamųjų variantai. Nuo to priklausys
parametrų interpretacija.


UŽDUOTIS. Reikia sudaryti logistinės regresijos modelį atsako kintamojo Y
vidurkiui modeliuoti ir įvertinti jo parametrus β₀ ir β₁ Newton-Raphson
metodu. Paiškinti gautų parametrų prasmę.


PASTABA. Gauti parametrų įverčiai turi sutapti su įverčiais, gautais naudojant 
standartinę R funkciją glm:


<code>mod = glm(Y ~ 1 + X, data = duomenys, family = binomial("logit")) </code>

<code>summary(mod)</code>



Duomenų įkėlimas.

In [4]:
d = read.csv2("ksimpah.csv", encoding="UTF-8")
head(d)

,Lytis,Hipertenzija,Tabakas,Alkoholis,Ūgis,Svoris,KMI,Antsvoris,EQ,EZ,FC,FE
,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1,Vyras,Sveikas,Nerūko,Geria,181.0,77.0,23.50,Ne,519.5,803.0,1.20,4.62
2,Vyras,Sveikas,Nerūko,Geria,182.0,108.0,32.60,Yra,517.5,797.0,2.15,5.22
3,Vyras,Serga,Nerūko,Negeria,187.0,122.0,34.89,Yra,654.0,554.5,2.85,4.89
4,Vyras,Sveikas,Nerūko,Geria,178.5,64.0,20.09,Ne,518.5,727.5,1.40,4.56
5,Moteris,Sveikas,Rūko,Geria,170.0,57.5,19.90,Ne,472.5,662.5,1.80,4.01
6,Vyras,Sveikas,Nerūko,Geria,187.5,70.0,19.91,Ne,516.0,680.5,1.55,4.73


Sudarysime tiesini modelį: $$g(\mu) = X\beta.$$

Čia $\mu$ yra atsako kintamojo $Y$ vidurkis, $X$ yra plano matrica, o $\beta$ yra modelio parametrų vektorius. $Y$ yra pasiskirstęs pagal binominį dėsnį, sąryšio funkcija *g* yra *logit*, tokiu atvėju modelis yra užrašomas tokiu pavidalu:

$$logit(\mu) = ln\Big(\frac{\mu}{1-\mu}\Big) = \eta = X\beta$$

**Newtono-Raphson metodas logistinės regresijos modeliui.** Užrašysime logistinės regresijos modelio parametrų vertinimo algoritmą. Šiuo atvėju saryšio funkcija *g* yra *logit*, todėl vidurkio išraiška yra tokia:
$$\mu = \frac{e^\eta}{1+e^\eta}$$

1. Pradiniai modelio $\beta$ parametrai yra generuojami atsitiktinai.

2. Pasinaudojant šia išraišką ir žinant predikatoriaus $\eta$ išraišką, gauname, kad *m-osios* iteracijos vidurkis skaičiuojamas pagal formulę: 

$$\mu = \frac{e^{X\beta^{(m)}}}{1+e^{X\beta^{(m)}}}$$

3. Turint vidurkius $\mu_i$, apskaičiuojame $Var(y_i)$ ir sudarome matricą $W$

$$Var(y_i) = \mu_i(1-\mu_i)$$

$$w_{ii} = \frac{1}{Var(y_i)} \Big(\frac{\partial \mu_i}{\partial \eta_i}\Big)^2 = \frac{1}{Var(y_i)} \Big(\frac{Var(y_i)}{a(\phi)}\Big)^2 = Var(y_i) = \mu_i(1-\mu_i).$$

4. Turint matricą $W$, apskaičiuojame naujus parametrų $\beta$ įverčois

$$\beta^{m+1} = \beta^{m} + \Big(X^TWX)^{-1}  X^T (y-\mu)\Big).$$

5. Kartojame ciklą nuo 2 žingsnio.

Skaičiavimai nutraukiami, kai skirtumas $\Delta = |\beta^{m+1} - \beta^m|$ pasidaro mažesni už tam tikrą iš anksto užsiduotą tikslumo lygį $\epsilon \ge 0$


Sudarome modelio plano matrica $X$:
$$\quad X=\begin{bmatrix}
1 & x_{11} & x_{12} & \dots & x_{1k}\\
1 & x_{21} & x_{22} & \dots & x_{2k} \\
\vdots  & \vdots & \vdots &   & \vdots\\ 
1 & x_{n1} & x_{n2} & \dots & x_{nk}
\end{bmatrix}$$


*KMI* stulpelį imame kaip regresorių $X$, duomenis normalizuosime:
$$ z = \frac{x-\mu}{\sigma}$$

In [5]:
normalizavimas = function(x) {
    z = (x - mean(x)) / sd(x)
    
    return(z)
}

Normalizuojame KMI stulpelį ir sudarome $X$ matricą pridedami stulpelį sudarytą iš vinetų.

In [105]:
kmi = unlist(matrix(d[c("KMI")]))
kmi_norm = matrix(normalizavimas(kmi))


X = cbind(rep(c(1), nrow(kmi_norm)), kmi_norm)
    
head(X)

1,0.1354655
1,1.8731536
1,2.3104400
1,-0.5156901
1,-0.5519715
1,-0.5500620


*Alkoholis* stulpelį pasirenkame kaip atsako kintamąjį $Y, $
$\quad Y=\begin{bmatrix}y_1 \\ y_2 \\ \vdots \\ y_n\end{bmatrix} \;$

$y_i = 0$ – žmogus geria

$y_i = 1$ – žmogus negeria


In [106]:
Y = as.numeric(unlist(d[c("Alkoholis")])) - 1
head(matrix(Y))

0
0
1
0
0
0


Atsitiktinai sugeneruokime pradinias parametro $\beta$ reikšmes.

In [107]:
beta_pradinis = matrix(rnorm(2), 2, 1)
beta_pradinis

0.6705745
-2.6140047


Sekantis žingsnis būtų $\mu$ vektoriaus skaičiavimas $\quad \mu=\begin{bmatrix}\mu_1 \\ \mu_2 \\ \vdots \\ \mu_n\end{bmatrix} \;$ 


pagal formulę $$\mu = \frac{e^\eta}{1+e^\eta} = \frac{e^{X\beta^{(m)}}}{1+e^{X\beta^{(m)}}}$$

Tam pasirašysime funkciją.

In [108]:
mu = function(X, beta) {
    eta = X %*% beta
    return(exp(eta)/(1+exp(eta)))
}

In [109]:
miu = mu(X, beta_pradinis)
head(miu)

0.578462990
0.014402337
0.004637524
0.882735224
0.892201727
0.891720708


Sekančiame žingsnyje mums reikia sudaryti diagonalinę svorių matricą $W$. Šiai matricai sudaryti taip pat pasirašysime funkciją, kadangi metodas iteracinis, teks skaičiuoti bent kelis kartus.

$$\quad W=\begin{bmatrix}
w_{11} & 0 & 0 & \dots & 0\\
0 & w_{22} & 0 & \dots & 0 \\
\vdots  & \vdots & \vdots &   & \vdots\\ 
0 & 0 & 0 & \dots & w_{nn}
\end{bmatrix}$$


$$w_{ii} = Var(y_i) = \mu_i(1-\mu_i)$$

In [110]:
W = function(miu) {
    diag(c(miu*(1-miu)))
}

In [111]:
head(W(miu))

0.2438436,0.00000000,0.000000000,0.0000000,0.00000000,0.00000000,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0.0000000,0.01419491,0.000000000,0.0000000,0.00000000,0.00000000,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0.0000000,0.00000000,0.004616018,0.0000000,0.00000000,0.00000000,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0.0000000,0.00000000,0.000000000,0.1035137,0.00000000,0.00000000,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0.0000000,0.00000000,0.000000000,0.0000000,0.09617781,0.00000000,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0.0000000,0.00000000,0.000000000,0.0000000,0.00000000,0.09655489,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


Turint matricą $W$, apskaičiuojame naujus parametrų $\beta$ įverčois

$$\beta^{m+1} = \beta^{m} + \Big(X^TWX)^{-1}  X^T (y-\mu)\Big).$$


In [112]:
beta_naujas = function(X, Y, beta) {
    miu = mu(X, beta)
    W. = W(miu)
    beta_n = beta + (solve(t(X) %*% W. %*% X) %*% t(X) %*% (Y-miu))
    
    return(beta_n)
}

In [113]:
beta_naujas(X,Y, beta_pradinis)

-2.366567
9.595901


Tikriname ar yra tenkinama sąlyga:
$$\Delta = |\beta^{m+1} - \beta^{m}| < \epsilon$$

Jeigune ne, tai skaičiavimai kartojami nuo $\mu$ perskaičiavimo žingsnio.

In [114]:
delta = function(beta_senas, beta_naujas, epsilon=1e-5) {
    
    sqrt(sum((beta_senas - beta_naujas) ^ 2)) > epsilon
}

Apjunkime visus šiuos žingsnius į vieną funkciją ir įteruokime kol pasieksime mūsų norimą tikslumą, toliau gautus $\beta$ koeficientus palyginsime su <code>glm</code> funkcija.

In [115]:
Newton_Raphson_metodas = function(X, Y, epsilon=1e-10) {
    # --------------------------------
    # X - plano matrica
    # Y - atsako kintamasis
    # epsilon - stabdymo kriterijus, pagal nutylejima epsilon=1e-5
    # --------------------------------
    
    beta_pradinis = matrix(rnorm(2), 2, 1)
    
    while (TRUE) {
        beta_n = beta_naujas(X,Y, beta_pradinis)
        delta_ = delta(beta_pradinis, beta_n, epsilon = epsilon)
        if (delta_) {
            cat("\nsu beta reikšmėmis:\n", "b1: ", beta_n[1], "\n", "b2: ", 
                beta_n[2], "\nsalyga neispildyta\n______________\n")
            beta_pradinis = beta_n
            
        } else {
            cat("\nSALYGA ISPILDYTA!")
            return(beta_n)}
        
    }
    
}

In [116]:
Newton_Raphson_metodas(X,Y)


su beta reikšmėmis:
 b1:  -1.225757 
 b2:  0.3153335 
salyga neispildyta
______________

su beta reikšmėmis:
 b1:  -1.221818 
 b2:  0.4466433 
salyga neispildyta
______________

su beta reikšmėmis:
 b1:  -1.22407 
 b2:  0.4467709 
salyga neispildyta
______________

su beta reikšmėmis:
 b1:  -1.224071 
 b2:  0.4467715 
salyga neispildyta
______________

SALYGA ISPILDYTA!

-1.2240712
0.4467715


palyginkime gautas $\beta$ reikšmes su <code> glm </code> funkcija.

In [121]:
mod = glm(Y ~ 1 + X, data = data.frame(X, Y), family = binomial("logit"))
summary(mod)


Call:
glm(formula = Y ~ 1 + X, family = binomial("logit"), data = data.frame(X, 
    Y))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2500  -0.7253  -0.6432  -0.5774   1.9789  

Coefficients: (1 not defined because of singularities)
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -1.2241     0.1977  -6.192 5.95e-10 ***
X1                NA         NA      NA       NA    
X2            0.4468     0.1790   2.496   0.0125 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 166.95  on 152  degrees of freedom
Residual deviance: 160.69  on 151  degrees of freedom
AIC: 164.69

Number of Fisher Scoring iterations: 4


Matome, kad rezultatai gauti iteraciniu Newton Raphson metodu sutampa su  *R* <code>glm</code> funkcijos rezultatais.